## Mosaic of the province of South-Tyrol

In [ ]:
# builtins
import os
import pathlib

# externals
from osgeo import gdal

# locals
from ai4ebv.core.constants import STT_TILES
from ai4ebv.core.utils import mosaic_tiles
from pysegcnn.core.utils import hdf2tifs, extract_by_mask

In [ ]:
# path to the HLS dataset
HLS_PATH = pathlib.Path('/mnt/CEPH_PROJECTS/AI4EBV/EO/HLS/')

In [ ]:
# reference year
YEAR = 2018

In [ ]:
# define sensor
SENSOR = 'S30'

In [ ]:
# shapefile defining province of South-Tyrol
ST = pathlib.Path('/mnt/CEPH_PROJECTS/AI4EBV/BOUNDARIES/ST.shp')

### Find scenes with least cloud cover

In [ ]:
scenes = {}
for tile in STT_TILES:
    scenes[tile] = {}
    for dirpath, dirname, files in os.walk(HLS_PATH.joinpath(tile, str(YEAR), SENSOR)):
        if files:
            files = sorted(files)
            for file in files:
                if file.endswith('hdf'):
                    path = os.sep.join([dirpath, file])
                    metadata = gdal.Info(path, format='json')['metadata']['']
                    cc = metadata.get('cloud_coverage')
                    sc = metadata.get('spatial_coverage')
                    if int(sc) == 100:
                        scenes[tile][path] = int(cc) if cc is not None else None
                        print(path, sc, cc)

In [ ]:
least_cc = {}
for k, v in scenes.items():
    least_cc[k] = {None: 100}
    for path, cc in v.items():
        least_cc[k] = {path: cc} if cc < list(least_cc[k].values()).pop() else least_cc[k]

In [ ]:
least_cc

In [ ]:
# convert hdf2tif
for tile in STT_TILES:
    hdf = pathlib.Path(list(least_cc[tile].keys()).pop())
    hdf2tifs(hdf, './', create_stack=False)

In [ ]:
# select a band
band = 'b02' if SENSOR == 'S30' else 'band02' 
tiles = []
for dirpath in pathlib.Path('./').iterdir():
    if dirpath.is_dir() and dirpath.name.startswith('HLS'):
        file = pathlib.Path('./').joinpath(dirpath, '_'.join([dirpath.name.replace('_', '.'), band]) + '.tif')
        tiles.append(file)
tiles

In [ ]:
# create mosaic
mosaic = mosaic_tiles(tiles, targets='./Mosaic.tif')

In [ ]:
# clip to province of South-Tyrol
extract_by_mask('./Mosaic.tif', ST, './Figures/Mosaic_clip.tif', overwrite=True)

In [ ]:
!rm -r ./HLS*

In [ ]:
!rm ./Mosaic.tif